In [1]:
import os
import math
from abc import abstractmethod

from PIL import Image
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
from torch import rand
from torchmetrics.image.fid import FrechetInceptionDistance
fid = FrechetInceptionDistance(feature=64)
# generate two slightly overlapping image intensity distributions
imgs_dist1 = torch.randint(0, 200, (100, 3, 28, 28), dtype=torch.uint8)
imgs_dist2 = torch.randint(100, 255, (100, 3, 28, 28), dtype=torch.uint8)
fid.update(imgs_dist1, real=True)
fid.update(imgs_dist2, real=False)
f = fid.compute()
print('done')

done


In [1]:
from utils import *
model = UNetModel(
    in_channels=3,
    model_channels=96,
    out_channels=3,
    channel_mult=(1, 2, 2),
    attention_resolutions=[]
)
nparams, fid, losslog, imgs, model = run(model, name='test', dataset_name='cifar10', epochs=1, batchsize=64, timesteps=5, device='cpu', test=True)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (utils.py, line 406)

In [1]:
import torch
import torch.nn as nn
from utils import *

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = UnetDWT2()
inp = torch.randn(10,3,32,32)
op = model(inp, torch.randn(10))
op.shape

0 cpu
0 cpu
1 cpu
1 cpu
2 cpu
2 cpu
hs cpu
hs cpu
hs cpu
hs cpu
hs cpu
hs cpu
hs cpu
hs cpu
hs cpu


torch.Size([10, 3, 32, 32])

In [ ]:


batch_size = 64
timesteps = 500

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# use MNIST dataset
dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# define model and diffusion
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'{device=}')
model = UNetModel(
    in_channels=1,
    model_channels=96,
    out_channels=1,
    channel_mult=(1, 2, 2),
    attention_resolutions=[]
)
# model = WaveMixSR(
#     depth = 4,
#     mult = 1,
#     ff_channel = 144,
#     final_dim = 144,
#     dropout = 0.3
# )
model.to(device)

gaussian_diffusion = GaussianDiffusion(timesteps=timesteps)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
print(sum([p.numel() for p in model.parameters()])/10**6, ' M')

In [1]:
from utils import *

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
a = Level1Waveblock( mult = 1,
 ff_channel = 16,
 final_dim = 96)
inp = torch.randn(10,96,32,32)
op = a(inp)
op.shape

torch.Size([10, 96, 32, 32])

In [2]:
from utils import *

dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())

In [5]:
dataset.data[:500].transpose((0,3,1,2)).shape

(500, 3, 32, 32)

In [18]:
sum([p.numel() for p in a.parameters()])

38056

In [4]:
b = AttentionBlock(channels=96)
sum([p.numel() for p in b.parameters()])

37152

In [12]:
dataset = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
gaussian_diffusion = GaussianDiffusion(timesteps=500)
corpus = 500
i1 = dataset.data[:corpus]
if i1.dim()==3 : i1 = i1.unsqueeze(1)
gen_imgs = gaussian_diffusion.sample(model, 28, batch_size=corpus, channels=1)
i2 = gen_imgs[-1].reshape(-1, 1, 28, 28)
i2 = torch.tensor(i2)
fid = calculate_fid(i1, i2)

sampling loop time step:   1%|▏         | 7/500 [01:22<1:36:23, 11.73s/it]


KeyboardInterrupt: 

In [8]:
from sortedcontainers import SortedSet

In [9]:
a = [1,2,3]
a.sort()

SortedSet([1, 2, 3])

In [11]:

a.add(4)
a.add(0)
a.add(1)
a

SortedSet([0, 1, 2, 3, 4])

In [3]:
@torch.jit.script
def _idwt1d(
    lo: torch.Tensor, hi: torch.Tensor, lo_hi: torch.Tensor, dim: int = -1
) -> torch.Tensor:
    """performs a 1d idwt on the defined dimension

    Args:
        lo (torch.Tensor): 5d low (average) coefs of shape [N,C,D,H,W]
        hi (torch.Tensor): 5d hi (detail) coefs of shape [N,C,D,H,W]
        lo_hi (torch.Tensor): lo,hi pass filters of shape [2,K]
        dim (int, optional): dimension to apply the idwt to . Defaults to -1.

    Returns:
        torch.Tensor: reconstructed tensor of shape [N,C,D_out,H_out,W_out] (e.g. H_out = 2*H if dim==-1)
    """
    dim = dim % 5

    groups = lo.shape[1]
    filter_c = (
        lo_hi[:, None, None, None, None, :]
        .repeat(1, groups, 1, 1, 1, 1)
        .swapaxes(5, dim + 1)  # swap filter to dwt dim
    )

    # stride of 2 for dwt dim
    stride = [1, 1, 1]
    stride[dim - 2] = 2
    padding = [0, 0, 0]
    padding[dim - 2] = lo_hi.shape[-1] - 2

    a_coefs = F.conv_transpose3d(
        lo, filter_c[0], stride=stride, padding=padding, groups=groups
    )
    d_coefs = F.conv_transpose3d(
        hi, filter_c[1], stride=stride, padding=padding, groups=groups
    )
    return a_coefs + d_coefs


@torch.jit.script
def _dwt1d(x: torch.Tensor, lo_hi: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """performs a 1d dwt on the defined dimension

    Args:
        x (torch.Tensor): 4d tensor of shape [N,C,D,H,W]
        lo_hi (torch.Tensor): low and highpass filter (shape [2,K])
        dim (int, optional): dimension to apply the dwt to . Defaults to -1.

    Returns:
        torch.Tensor: dwt coefs of shape [N,2,C,D_out,H_out,W_out]. The average and detail coefs are concatenated in the channels
    """
    dim = dim % 5
    groups = x.shape[1]
    # repeat filter to match number of channels
    filter_c = lo_hi[:, None, None, None, :].repeat(groups, 1, 1, 1, 1).swapaxes(4, dim)

    if x.shape[dim] % 2 != 0:
        # pad dwt dimension to multiple of two
        pad = [0] * 6
        pad[(4 - dim) * 2 + 1] = 1
        x = F.pad(x, pad)

    # stride of 2 for dwt dim
    stride = [1, 1, 1]
    stride[dim - 2] = 2

    padding = [0, 0, 0]
    padding[dim - 2] = lo_hi.shape[-1] - 2

    filtered = F.conv3d(x, filter_c, stride=stride, padding=padding, groups=groups)
    return filtered.reshape(
        filtered.shape[0],
        groups,
        2,
        filtered.shape[2],
        filtered.shape[3],
        filtered.shape[4],
    ).swapaxes(1, 2)


@torch.jit.script
def _dwt2(x: torch.Tensor, lohi: torch.Tensor) -> torch.Tensor:
    lh = _dwt1d(x[:, :, None, :, :], lohi, -1)
    y = _dwt1d(lh.flatten(1, 2), lohi, -2).squeeze(-3)

    # reorder coefs to match pywt ordering
    return y.reshape(x.shape[0], 4, x.shape[1], y.shape[-2], y.shape[-1])


@torch.jit.script
def _idwt2(x: torch.Tensor, lohi: torch.Tensor) -> torch.Tensor:
    ll, hl, lh, hh = x.swapaxes(1, 2).unsqueeze(-3).unbind(2)

    lo = _idwt1d(ll, lh, lohi, dim=-2)

    hi = _idwt1d(hl, hh, lohi, dim=-2)
    y = _idwt1d(lo, hi, lohi, dim=-1)
    return y.squeeze(-3)


@torch.jit.script
def _dwt3(x: torch.Tensor, lohi: torch.Tensor) -> torch.Tensor:
    x_c = _dwt1d(x, lohi, -1)
    y_c = _dwt1d(x_c.flatten(1, 2), lohi, -2)
    z_c = _dwt1d(y_c.flatten(1, 2), lohi, -3)
    return z_c.reshape(
        z_c.shape[0], 8, x.shape[1], z_c.shape[3], z_c.shape[4], z_c.shape[5]
    )


@torch.jit.script
def _idwt3(x: torch.Tensor, lohi: torch.Tensor) -> torch.Tensor:
    lll, hll, lhl, hhl, llh, hlh, lhh, hhh = x.unbind(1)

    ll = _idwt1d(lll, llh, lohi, dim=-3)
    hl = _idwt1d(hll, hlh, lohi, dim=-3)

    lh = _idwt1d(lhl, lhh, lohi, dim=-3)
    hh = _idwt1d(hhl, hhh, lohi, dim=-3)

    l = _idwt1d(ll, lh, lohi, dim=-2)
    h = _idwt1d(hl, hh, lohi, dim=-2)
    y = _idwt1d(l, h, lohi, dim=-1)
    return y

In [4]:
from pywt import Wavelet

def _to_wavelet_coefs(wavelet: str | torch.Tensor | Wavelet) -> torch.Tensor:
    match wavelet:
        case str():
            return torch.tensor(Wavelet(wavelet).filter_bank)[2:]
        case torch.Tensor():
            return wavelet
        case Wavelet():
            return torch.tensor(wavelet.filter_bank)[2:]
        case _:
            raise Exception("")

def dwt2(x: torch.Tensor, wavelet: str | torch.Tensor | Wavelet) -> torch.Tensor:
    """performs the 2D discrete wavelet transform

    Args:
        x (torch.Tensor): [N,C,H,W] data
        wavelet (str | torch.Tensor | Wavelet): wavelet

    Returns:
        torch.Tensor: dwt coefs of shape [M,4,C,H_out,W_out] \\
            (coef oder: cA,cV,cW,cD)
    """
    filter = _to_wavelet_coefs(wavelet).to(x.device)
    return _dwt2(x, filter)


def idwt2(x: torch.Tensor, wavelet: str | torch.Tensor | Wavelet) -> torch.Tensor:
    """performs the 2D inverse discrete wavelet transform

    Args:
        x (torch.Tensor): [N,4,C,H,W] average and detail coefs
        wavelet (str | torch.Tensor | Wavelet): wavelet

    Returns:
        torch.Tensor: reconstructed tensor
    """
    lohi = _to_wavelet_coefs(wavelet).to(x.device)
    return _idwt2(x, lohi)

In [1]:
from utils import *

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_up = InverseWaveDownsample(channels=3)
inp = torch.randn((64, 3, 32, 32))
op = model_up(inp)
inp.shape, op.shape

(torch.Size([64, 3, 32, 32]), torch.Size([64, 3, 64, 64]))

In [5]:
# db stands for Daubechies 
# check https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html

x = torch.rand(8,3,100,100)
coefs = dwt2(x,"db2") # coefs of shape (1,2,3,50)
# reconstruct signal from coefficients
y = idwt2(coefs,"db2")

In [ ]:
# using haar wavelets
x = torch.rand(8,3,100,100)
coefs = dwt2(x,"haar") # coefs of shape (1,2,3,50)
# reconstruct signal from coefficients
y = idwt2(coefs,"haar")
x.shape, coefs.shape, y.shape

(torch.Size([8, 3, 100, 100]),
 torch.Size([8, 4, 3, 50, 50]),
 torch.Size([8, 3, 100, 100]))

In [10]:
coefs.shape, y.shape, torch.allclose(x, y), (x-y).abs().max()

(torch.Size([8, 4, 3, 51, 51]),
 torch.Size([8, 3, 100, 100]),
 False,
 tensor(2.9802e-07))

In [13]:
fcoefs = coefs.flatten(1,2)
fcoefs.shape

torch.Size([8, 12, 51, 51])

In [16]:
torch.equal(fcoefs[:,:3], coefs[:,0]), torch.equal(fcoefs[:,3:6], coefs[:,1]), torch.equal(fcoefs[:,6:9], coefs[:,2]), torch.equal(fcoefs[:,9:], coefs[:,3])

(True, True, True, True)

In [93]:
from utils import *
xf1 = DWTForward(J=1, mode='zero', wave='db1') 

class WaveDownsample(nn.Module) :
    def __init__(self, mult = 2, channels=1, final_dim = 16, dropout = 0.5,) :
        super().__init__()
        self.feedforward = nn.Sequential(
                nn.Conv2d(final_dim, final_dim*mult,1),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Conv2d(final_dim*mult, final_dim, 1),
                nn.Conv2d(final_dim, channels, 3, 1, 1),
                nn.BatchNorm2d(channels)
            
            )

        self.reduction = nn.Conv2d(final_dim, int(final_dim/4), 1)
        self.pre = nn.Sequential(
            nn.Conv2d(channels, int(final_dim/2), 3, 1, 1),
            nn.Conv2d(int(final_dim/2), final_dim, 3, 1, 1)
        )
    def forward(self, x) :
        x = self.pre(x)
        b, c, h, w = x.shape
        x = self.reduction(x) # c/4, h, w
        Y1, Yh = xf1.to(x.device)(x)
        x = torch.reshape(Yh[0], (b, int(c*3/4), int(h/2), int(w/2)))
        x = torch.cat((Y1,x), dim = 1) # c, h/2, w/2
        return self.feedforward(x)
    
class InverseWaveDownsample(nn.Module) :
    def __init__(self, mult = 2, channels=1, final_dim = 16, dropout = 0.5,) :
        super().__init__()
        self.feedforward = nn.Sequential(
                nn.Conv2d(channels, final_dim, 3, 1, 1),
                nn.Conv2d(final_dim, final_dim*mult, 1),
                nn.BatchNorm2d(final_dim*mult),
                nn.Conv2d(final_dim*mult, final_dim,1),
                nn.GELU(),
                nn.Dropout(dropout),
            
            )

        self.reduction = nn.Conv2d(final_dim, final_dim*4, 1)
        self.pre = nn.Sequential(
            nn.Conv2d(final_dim, int(final_dim/2), 3, 1, 1),
            nn.Conv2d(int(final_dim/2), channels, 3, 1, 1)
        )
    def forward(self, x) :
        x = self.feedforward(x)
        b, c, h, w = x.shape
        x = self.reduction(x) # 4*c, h, w
        temp = x.chunk(4, dim=1)
        coefs = torch.stack(temp).transpose(0,1)
        x = idwt2(coefs,"db1") # c,2*h,2*w
        return self.pre(x)

In [37]:
a = torch.randn((64,12,32,32)).chunk(4, dim=1)
torch.stack(a).transpose(0,1).shape

torch.Size([64, 4, 3, 32, 32])

In [41]:
inp = torch.randn((64,12,32,32))
temp = inp.chunk(4, dim=1)
coefs = torch.stack(temp).transpose(0,1)
y = idwt2(coefs,"haar")
inp.shape, y.shape

(torch.Size([64, 12, 32, 32]), torch.Size([64, 3, 64, 64]))

In [94]:
model_down = WaveDownsample(channels=3)
inp = torch.randn((64, 3, 32, 32))
op = model_down(inp)
inp.shape, op.shape

(torch.Size([64, 3, 32, 32]), torch.Size([64, 3, 16, 16]))

In [95]:
model_up = InverseWaveDownsample(channels=3)
inp = torch.randn((64, 3, 32, 32))
op = model_up(inp)
inp.shape, op.shape

(torch.Size([64, 3, 32, 32]), torch.Size([64, 3, 64, 64]))

In [88]:
inp = torch.randn((64,3,32,32))
b, c, h, w = inp.shape
Y1, Yh = xf1(inp)
mid = torch.reshape(Yh[0], (b, int(c*3), int(h/2), int(w/2)))
x=mid = torch.cat((Y1,mid), dim = 1) # c, h/2, w/2
inp.shape, mid.shape

(torch.Size([64, 3, 32, 32]), torch.Size([64, 12, 16, 16]))

In [89]:
temp = mid.chunk(4, dim=1)
coefs = torch.stack(temp).transpose(0,1)
y = idwt2(coefs,"db1")
mid.shape, y.shape

(torch.Size([64, 12, 16, 16]), torch.Size([64, 3, 32, 32]))

In [90]:
torch.allclose(inp, y), (inp-y).abs().max()

(False, tensor(5.9326))

In [91]:
mid2 = dwt2(inp, 'db1').flatten(1,2)
mid.shape, mid2.shape

(torch.Size([64, 12, 16, 16]), torch.Size([64, 12, 16, 16]))

In [92]:
(mid-mid2).abs().max()

tensor(6.8996)

In [4]:
from utils import *

model = WaveUNetModel(
    in_channels=3,
    model_channels=96,
    out_channels=3,
    use_idwt=True,
    channel_mult=(1, 2, 2),
    attention_resolutions=[]
)

In [ ]:
a = [1,2,3,4]


'4321'

In [8]:
from collections import deque

a = deque([6, 7, 8, 9])
''.join(map(str, reversed(a)))

'9876'

In [1]:
from utils import *
from datasets import load_dataset

ds = load_dataset("zh-plus/tiny-imagenet")

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch
train_loader = torch.utils.data.DataLoader(ds['train'], batch_size=64, shuffle=False)

In [10]:
from torch.utils.data import Dataset
class TinyImageNet(Dataset) :
    def __init__(self, data, split='train', transforms=None) :
        assert split in ['train', 'test'], f'Split {split} not supported'
        self.data = data[split]
        self.transforms = transforms
    def __len__(self) :
        return len(self.data)
    def __getitem__(self, idx) :
        ret = self.transforms(np.array(self.data[idx]['image']))
        label = self.data[idx]['label']
        if ret.shape[-3]==1 :
            ret = torch.repeat_interleave(ret, repeats=3, dim=-3)
        return ret, label

In [60]:
temp = torch.randn((1,64,64))
torch.repeat_interleave(temp, repeats=3, dim=-3).shape

torch.Size([3, 64, 64])

In [2]:
from torchvision import transforms
from datasets import load_dataset

# ds = load_dataset("zh-plus/tiny-imagenet")
dataset = TinyImageNet(ds, transforms=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [12]:
next(iter(train_loader))[0].shape

torch.Size([64, 3, 64, 64])

In [16]:
imgs, labels = next(iter(train_loader))

In [17]:
imgs.shape, labels.shape

(torch.Size([64, 3, 64, 64]), torch.Size([64]))

In [18]:
labels

tensor([ 26, 145,  34,  88,  81,  20,  61,   1, 177,  79,  20,  54,  90,  67,
         37, 183, 161, 114,  31, 185, 158,  18, 123,  45,  60, 135,  47,  31,
         53,  69,  20, 102,   9, 195, 164,  57, 135, 146, 187, 128, 167,  42,
          8,  84, 108, 109, 135,  64, 195,  17, 122, 176, 132,  26, 183,  39,
          2, 145, 183,  67, 188, 143,  50,  51])

In [1]:
from utils import *

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from utils import *
model = UNetModel(
     in_channels=3,
    model_channels=96,
    out_channels=3,
    channel_mult=(1, 2, 2),
    attention_resolutions=[]

)
sum([p.numel() for p in model.parameters()])

c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13800003

In [ ]:
nparams, fid, losslog, imgs, model = run(model, name='test', dataset_name='tinyImageNet', epochs=5, batchsize=64, timesteps=500, device='cpu', test=True)

device='cpu'


  0%|          | 0/1563 [00:29<?, ?it/s]

Loss: 1.029611587524414


In [63]:
dataset = TinyImageNet(ds, transforms=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64)

for i, iter in enumerate(train_loader) :
    print(i, iter.shape)

0 torch.Size([64, 3, 64, 64])
1 torch.Size([64, 3, 64, 64])
2 torch.Size([64, 3, 64, 64])
3 torch.Size([64, 3, 64, 64])
4 torch.Size([64, 3, 64, 64])
5 torch.Size([64, 3, 64, 64])
6 torch.Size([64, 3, 64, 64])
7 torch.Size([64, 3, 64, 64])
8 torch.Size([64, 3, 64, 64])
9 torch.Size([64, 3, 64, 64])
10 torch.Size([64, 3, 64, 64])
11 torch.Size([64, 3, 64, 64])
12 torch.Size([64, 3, 64, 64])
13 torch.Size([64, 3, 64, 64])
14 torch.Size([64, 3, 64, 64])
15 torch.Size([64, 3, 64, 64])
16 torch.Size([64, 3, 64, 64])
17 torch.Size([64, 3, 64, 64])
18 torch.Size([64, 3, 64, 64])
19 torch.Size([64, 3, 64, 64])
20 torch.Size([64, 3, 64, 64])
21 torch.Size([64, 3, 64, 64])
22 torch.Size([64, 3, 64, 64])
23 torch.Size([64, 3, 64, 64])
24 torch.Size([64, 3, 64, 64])
25 torch.Size([64, 3, 64, 64])
26 torch.Size([64, 3, 64, 64])
27 torch.Size([64, 3, 64, 64])
28 torch.Size([64, 3, 64, 64])
29 torch.Size([64, 3, 64, 64])
30 torch.Size([64, 3, 64, 64])
31 torch.Size([64, 3, 64, 64])
32 torch.Size([64,

KeyboardInterrupt: 

In [ ]:
import optimagic as om
import math
import numpy as np

def func(x) :
    return math.tan(x)

lbfgsb_res = om.maximize(
    fun=func,
    params=0,
    algorithm="scipy_lbfgsb",
)
lbfgsb_res.params.round(5)

1.56467

In [24]:
import numpy as np
from optimagic import minimize
# from optimagic import parameter_bounds

def test_function(params):
    """A test function that produces infinite gradients at certain points."""
    x, y = params["x"], params["y"]
    
    # Function value
    f = x**2 + y**2
    
    # Gradient with potential infinite values
    # grad = {
    #     "x": 2 * x,
    #     "y": 2 * y / (y - 1)  # This will be infinite when y = 1
    # }
    
    return f #, grad

# Define parameters with bounds
params = {
    "x": 1.0,
    "y": 0.5  # Start close to y=1 to trigger infinite gradient
}

# bounds = parameter_bounds(
#     lower={"x": -10, "y": -10},
#     upper={"x": 10, "y": 10}
# )

# Test 1: With error_handling="raise" (should raise InvalidFunctionError)
result = minimize(
    criterion=test_function,
    params=params,
    algorithm="scipy_lbfgsb",
    # bounds=bounds,
    error_handling="raise"
)

print(result.params)

# Test 2: With error_handling="continue" (should use penalty values)
result = minimize(
    criterion=test_function,
    params=params,
    algorithm="scipy_lbfgsb",
    # bounds=bounds,
    error_handling="continue"
)
print("\nTest 2 (continue) - Optimization completed with penalty values")
print(f"Final parameters: {result.params}")
print(f"Final criterion value: {result.criterion}")

{'x': 4.991182883662404e-10, 'y': -6.638553873283826e-10}

Test 2 (continue) - Optimization completed with penalty values
Final parameters: {'x': 4.991182883662404e-10, 'y': -6.638553873283826e-10}
Final criterion value: 6.898230410665623e-19


c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\optimagic\deprecations.py:31: FutureWarning: To align optimagic with scipy.optimize, the `criterion` argument has been renamed to `fun`. Please use `fun` instead of `criterion`. Using `criterion`  will become an error in optimagic version 0.6.0 and later.
  warnings.warn(msg, FutureWarning)
c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\optimagic\deprecations.py:31: FutureWarning: To align optimagic with scipy.optimize, the `criterion` argument has been renamed to `fun`. Please use `fun` instead of `criterion`. Using `criterion`  will become an error in optimagic version 0.6.0 and later.
  warnings.warn(msg, FutureWarning)
c:\Users\azizs\AppData\Local\Programs\Python\Python312\Lib\site-packages\optimagic\optimization\optimize_result.py:77: FutureWarning: The criterion attribute is deprecated. Use the fun attribute instead.
  warnings.warn(msg, FutureWarning)


In [3]:
from helpers import *

In [8]:
x = torch.randn((32, 16, 64, 64))
patch_size = 4
embed = get_2d_sincos_pos_embed(128, (x.shape[-1]//patch_size))
embed.shape

torch.Size([256, 128])

In [4]:
import torch, torchvision
dataset = torchvision.datasets.MNIST(root='data/', train=True, transform=torchvision.transforms.ToTensor())

In [5]:
a = torch.randn((1,1,28,28))

In [10]:
type(a)

torch.Tensor

In [11]:
isinstance(a, torch.Tensor)

True

In [15]:
class CustomMNIST(torch.utils.data.Dataset) :
    def __init__(self, root, train, transform) :
        self.data = torchvision.datasets.MNIST(root=root, train=train, transform=transform)
    def __len__(self) :
        return len(self.data)
    def __getitem__(self, idx):
        imgs, labels = self.data[idx]
        return imgs.numpy(), labels

In [28]:
data = CustomMNIST(root='data/', train=True, transform=torchvision.transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)
dataset = iter(trainloader)

In [35]:
imgs, labels = next(dataset)

In [30]:
imgs.shape, type(imgs), labels

(torch.Size([32, 1, 28, 28]),
 torch.Tensor,
 tensor([1, 1, 1, 2, 0, 7, 0, 9, 1, 8, 3, 8, 5, 8, 5, 3, 2, 3, 1, 3, 4, 5, 0, 9,
         8, 2, 4, 0, 8, 7, 1, 1]))

In [1]:
from shortcut_models.model import *

In [2]:
# x = torch.randn((32,64,16,16))
# model = DiT(patch_size=4, hidden_size=128, depth=4, num_heads=2, mlp_ratio=4, out_channels=16, class_dropout_prob=0., num_classes=10)

# Instantiate the module
model = DiT(patch_size=4, hidden_size=128, depth=4, num_heads=2, mlp_ratio=4, out_channels=16, class_dropout_prob=0., num_classes=10)

# Initialize parameters
key = jax.random.PRNGKey(0)
dummy_input = jnp.ones((32,64,16,16))  # whatever input shape you need
dummy_t = jnp.ones((32,))  # whatever input shape you need
dummy_y = jnp.ones((32,), dtype='int32') # whatever input shape you need
params = model.init(key, dummy_input, dummy_t, dummy_t, dummy_y)  # Binds the module

# Apply the model
output = model.apply(params, dummy_input, dummy_t, dummy_t, dummy_y)

DiT: Input of shape (32, 16, 16, 64) dtype float32
x (32, 16, 16, 64)
DiT: After patch embed, shape is (32, 16, 128) dtype bfloat16
DiT: Patch Embed of shape (32, 16, 128) dtype bfloat16
DiT: Conditioning of shape (32, 128) dtype float32
DiT: Input of shape (32, 16, 16, 64) dtype float32
x (32, 16, 16, 64)
DiT: After patch embed, shape is (32, 16, 128) dtype bfloat16
DiT: Patch Embed of shape (32, 16, 128) dtype bfloat16
DiT: Conditioning of shape (32, 128) dtype float32


In [3]:
output.shape

(32, 16, 16, 16)

torch.Size([32, 1024, 16])